In [1]:
import sys
import os
print(f"Current Working Directory: {os.getcwd()}")
!pip install -r requirements.txt -q

Current Working Directory: /home/dtrautner/dev/ner-quality-impact

[notice] A new release of pip is available: 23.1.2 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from src.noise.semantic import semantic_noise
from src.noise.orthographic import typo_tokens
from src.noise.label_noise import apply_label_noise_on_spans
from src.noise.syntactic import syntactic_noise

from src.data_preprocessing import build_label_maps, load_conll2003

print("Setup complete. Modules imported successfully.")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The

Setup complete. Modules imported successfully.


In [3]:
ds = load_conll2003()
id2label, label2id = build_label_maps(ds["train"].features, "ner_tags")
id2pos, pos2id = build_label_maps(ds["train"].features, "pos_tags")

# Take two example sentences from the training set
train_ds = ds["train"]
num_examples = len(train_ds)
middle_index = num_examples // 2
examples = train_ds.select(range(middle_index, middle_index + 2))

print(f"Loaded {len(examples)} example sentences.")

Loaded 2 example sentences.


In [4]:
def print_tokens_with_labels(tokens, ner_labels):
    """A helper function to print tokens and NER labels in an aligned format."""
    token_lengths = [len(token) for token in tokens]

    token_line = " ".join(tokens)
    
    # Create the string for labels, padded to align with the tokens above
    label_line = " ".join([label.ljust(length) for label, length in zip(ner_labels, token_lengths)])
    
    print("Tokens: " + token_line)
    print("Labels: " + label_line)

In [5]:
def demonstrate_noise_effect(
    noise_function, 
    example, 
    id2label, 
    id2pos, 
    noise_params, 
):
    """
    A generic function to apply any noise function to an example and print the result.
    """
    original_tokens = example["tokens"]
    pos_tag_ids = example["pos_tags"]
    original_ner_tag_ids  = example["ner_tags"]
    
    pos_tags = [id2pos[tag_id] for tag_id in pos_tag_ids]
    original_ner_labels = [id2label[tag_id] for tag_id in original_ner_tag_ids ]

    print("=" * 60)
    print("Original Sentence:")
    print_tokens_with_labels(original_tokens, original_ner_labels)
    print("\n")
    
    func_name = noise_function.__name__
    noisy_tokens = original_tokens
    noisy_ner_labels = original_ner_labels

    if func_name == "semantic_noise":
        noisy_tokens = noise_function(
            tokens=original_tokens,
            pos_tags=pos_tags,
            ner_tags=original_ner_tag_ids ,
            id2label=id2label,
            **noise_params
        )
    elif func_name == "typo_tokens":
        noisy_tokens = noise_function(
            tokens=original_tokens,
            ner_tags=original_ner_tag_ids,
            id2label=id2label,
            **noise_params
        )
    elif func_name == "syntactic_noise":
        params_with_o_label = {"o_label": label2id["O"], **noise_params}
        new_tokens, new_ner_tag_ids = noise_function(
            original_tokens, original_ner_tag_ids, id2label=id2label, label2id=label2id, **params_with_o_label
        )
        noisy_tokens = new_tokens
        noisy_ner_labels = [id2label[tag_id] for tag_id in new_ner_tag_ids]
    elif "apply_label_noise_on_spans" in func_name: 
        new_ner_tag_ids = noise_function(
            original_ner_tag_ids, id2label, label2id, **noise_params
        )
        noisy_ner_labels = [id2label[tag_id] for tag_id in new_ner_tag_ids]
    else:
        # Fallback for other potential functions
        print(f"Warning: Don't know the specific signature for {func_name}. Trying a generic call.")
        noisy_tokens = noise_function(tokens=original_tokens, **noise_params)

    print(f"Sentence After '{func_name}':")
    print_tokens_with_labels(noisy_tokens, noisy_ner_labels)
    print("=" * 60)
    print("\n")

In [6]:

semantic_params = {
    "p": 0.6,
    "entity_strategy": "all",
    "ops": ["contextual"], #"synonym", "word_embs", "contextual"
}

index = 134
examples = train_ds.select(range(index, index + 2))

for example in examples:
    demonstrate_noise_effect(
            noise_function=semantic_noise,
            example=example,
            id2label=id2label,
            id2pos=id2pos,
            noise_params=semantic_params,
        )

Original Sentence:
Tokens: A spokesman for the group said the meeting " signals a new level of cooperation between Mujahideen Khalq and the Iranian Kurdish oppositions " .
Labels: O O         O   O   O     O    O   O       O O       O O   O     O  O           O       B-ORG      I-ORG O   O   B-MISC  I-MISC  O           O O


Loading contextual model: albert-base-v2


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForMaskedLM: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


Sentence After 'semantic_noise':
Tokens: A spokesman for the group said the announcement " opened a greater sort of relations between Mahmoud Khalq and the Iranian Political faction " .
Labels: O O         O   O   O     O    O   O            O O      O O       O    O  O         O       B-ORG   I-ORG O   O   B-MISC  I-MISC    O       O O


Original Sentence:
Tokens: Iran heavily bombarded targets in northern Iraq in July in pursuit of KDPI guerrillas based in Iraqi Kurdish areas outside the control of the government in Baghdad .
Labels: B-LOC O       O         O       O  O        B-LOC O  O    O  O       O  B-ORG O          O     O  B-MISC I-MISC  O     O       O   O       O  O   O          O  B-LOC   O


Sentence After 'semantic_noise':
Tokens: Kurds repeatedly bombed insurgents in northern Syria in July in favour of HEZBOLLAH guerrillas based in Shia Border provinces outside the control of the government in Baghdad .
Labels: B-LOC O          O      O          O  O        B-LOC O  O   

In [7]:
orthographic_params = { "p": 0.3, "protect_entities": False }

index = 134
examples = train_ds.select(range(index, index + 2))

for example in examples:
    demonstrate_noise_effect(
            noise_function=typo_tokens,
            example=example,
            id2label=id2label,
            id2pos=id2pos,
            noise_params=orthographic_params,
        )

Original Sentence:
Tokens: A spokesman for the group said the meeting " signals a new level of cooperation between Mujahideen Khalq and the Iranian Kurdish oppositions " .
Labels: O O         O   O   O     O    O   O       O O       O O   O     O  O           O       B-ORG      I-ORG O   O   B-MISC  I-MISC  O           O O


Sentence After 'typo_tokens':
Tokens: A spokesman for the grop sadi thw jmeeting " signals a new level of cooperation bdtween Myjahideen Khalq and the Iranian Kurdish oppositions " .
Labels: O O         O   O   O    O    O   O        O O       O O   O     O  O           O       B-ORG      I-ORG O   O   B-MISC  I-MISC  O           O O


Original Sentence:
Tokens: Iran heavily bombarded targets in northern Iraq in July in pursuit of KDPI guerrillas based in Iraqi Kurdish areas outside the control of the government in Baghdad .
Labels: B-LOC O       O         O       O  O        B-LOC O  O    O  O       O  B-ORG O          O     O  B-MISC I-MISC  O     O       O   O  

In [8]:
syntactic_params  = { "p": 0.3}

index = 134
examples = train_ds.select(range(index, index + 2))

for example in examples:
    demonstrate_noise_effect(
            noise_function=syntactic_noise,
            example=example,
            id2label=id2label,
            id2pos=id2pos,
            noise_params=syntactic_params,
        )

Original Sentence:
Tokens: A spokesman for the group said the meeting " signals a new level of cooperation between Mujahideen Khalq and the Iranian Kurdish oppositions " .
Labels: O O         O   O   O     O    O   O       O O       O O   O     O  O           O       B-ORG      I-ORG O   O   B-MISC  I-MISC  O           O O


Sentence After 'syntactic_noise':
Tokens: A : spokesman for the group said the meeting " signals new , level of cooperation ; between Mujahideen Khalq and the Iranian Kurdish oppositions , " .
Labels: O O O         O   O   O     O    O   O       O O       O   O O     O  O           O O       B-ORG      I-ORG O   O   B-MISC  I-MISC  O           O O O


Original Sentence:
Tokens: Iran heavily bombarded targets in northern Iraq in July in pursuit of KDPI guerrillas based in Iraqi Kurdish areas outside the control of the government in Baghdad .
Labels: B-LOC O       O         O       O  O        B-LOC O  O    O  O       O  B-ORG O          O     O  B-MISC I-MISC  O    

In [9]:
label_params  = { "p": 0.50, "to_O_prob": 0.50 }

index = 134
examples = train_ds.select(range(index, index + 2))

for example in examples:
    demonstrate_noise_effect(
            noise_function=apply_label_noise_on_spans,
            example=example,
            id2label=id2label,
            id2pos=id2pos,
            noise_params=label_params,
        )

Original Sentence:
Tokens: A spokesman for the group said the meeting " signals a new level of cooperation between Mujahideen Khalq and the Iranian Kurdish oppositions " .
Labels: O O         O   O   O     O    O   O       O O       O O   O     O  O           O       B-ORG      I-ORG O   O   B-MISC  I-MISC  O           O O


Sentence After 'apply_label_noise_on_spans':
Tokens: A spokesman for the group said the meeting " signals a new level of cooperation between Mujahideen Khalq and the Iranian Kurdish oppositions " .
Labels: O O         O   O   O     O    O   O       O O       O O   O     O  O           O       O          O     O   O   B-MISC  I-MISC  O           O O


Original Sentence:
Tokens: Iran heavily bombarded targets in northern Iraq in July in pursuit of KDPI guerrillas based in Iraqi Kurdish areas outside the control of the government in Baghdad .
Labels: B-LOC O       O         O       O  O        B-LOC O  O    O  O       O  B-ORG O          O     O  B-MISC I-MISC  O     